In [3]:
import pandas as pd

# Note there are 6 tasks, each of which have their own CSV file. 
# Each CSV has 3 columns for 3/4 prompts: basic_prompt, instructive_prompt, CoT_prompt
#   The 4th prompt, post-instructive, consists of
#       basic_prompt -> basic_answer
#       basic_answer + instructive_prompt -> revised_answer
# Each CSV has 100 rows (100 questions per task)

task_names = [  
                'common_problem_task_prompt',
                'consequences_task_prompt',
                'im_task_prompt',
                'js_task_prompt',
                'situation_task_prompt',
                'unusual_task_prompt',
            ]

for t in task_names:    
    df = pd.read_csv(f'raw/{t}.csv')
    print(df.shape)
    # df.to_json(orient='records', indent=4)




(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)


In [5]:
import pandas as pd
import json

df_common_problem = pd.read_csv('raw/common_problem_task_prompt.csv')
df_consequences = pd.read_csv('raw/consequences_task_prompt.csv')
df_im = pd.read_csv('raw/im_task_prompt.csv')
df_is = pd.read_csv('raw/is_task_prompt.csv')
df_js = pd.read_csv('raw/js_task_prompt.csv')
df_situation = pd.read_csv('raw/situation_task_prompt.csv')
df_unusual = pd.read_csv('raw/unusual_task_prompt.csv')

dfs = [('common_problem',df_common_problem), 
       ('consequences',df_consequences), 
       ('im',df_im), 
       ('is',df_is), 
       ('js',df_js), 
       ('situation',df_situation), 
       ('unusual',df_unusual)]

# Define personas for "constraints"
# personas = [
#     "Do the following task or answer the following question.", 
#     "Act like a typical engineer. Do the following task or answer the following question.",
#     "Act like a typical farmer. Do the following task or answer the following question.",
#     "Act like a typical merchant. Do the following task or answer the following question.",
#     "Act like a typical music artist. Do the following task or answer the following question.",
#     "Act like a typical natural scientist. Do the following task or answer the following question.",
#     "Act like a typical primary school student. Do the following task or answer the following question."
# ]

# Transform DataFrame into desired JSON format
json_data = []
for df_name, df in dfs:
    for idx, row in df.iterrows():
        entry = {
            "meta_data": {
                "dataset": "ttct",
                "id": f'{idx+1}_{df_name}',
                "eval_func": None
            },
            "input": {
                "text_basic": row['basic_prompt'],
                "text_instructive": row['instructive_prompt'],
                "text_cot": row['CoT_prompt'],
                "file": None,
                "others": {
                    "constraints": None,
                    "instructions": {
                        "system": "Do the following task or answer the following question.",
                        "user": "Instruction: {question}"
                    },
                    "references": {
                        "solutions": None
                    }
                }
            },
            "output": None
        }
        json_data.append(entry)

# Save JSON file
with open('processed/ttct.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=4, ensure_ascii=False)

print("JSON file successfully created.")


JSON file successfully created.


In [1]:
import json

with open('processed/ttct.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Recursively rename 'text_CoT' to 'text_cot'
def rename_keys(obj):
    if isinstance(obj, dict):
        return {('text_cot' if k == 'text_CoT' else k): rename_keys(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [rename_keys(elem) for elem in obj]
    else:
        return obj

updated_data = rename_keys(data)

# Save the updated JSON
with open('processed/ttct2.json', 'w', encoding='utf-8') as f:
    json.dump(updated_data, f, indent=4, ensure_ascii=False)


In [2]:
import json

with open('processed/ttct-old2.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Remove the constraints from each entry
for entry in data:
    if "others" in entry and "constraints" in entry["others"]:
        entry["others"]["constraints"] = []

with open('ttct.json', 'w') as file:
    json.dump(data, file, indent=4)

print(json.dumps(data, indent=4))


[
    {
        "meta_data": {
            "dataset": "ttct",
            "id": "1_common_problem",
            "eval_func": null
        },
        "input": {
            "text_basic": "Common Problems Task. In this task, you will be presented with a scenario or situation. Your job is to think about it and identify as many potential problems or issues that may arise in connection with each situation. The scenario is: Planning a birthday party for a 5-year-old.",
            "text_instructive": "Common Problems Task. In this task, you will be presented with a scenario or situation. Your job is to think about it and identify as many potential problems or issues that may arise in connection with the situation. There's no right or wrong answers, we're interested in how many different problems you can identify and the variety of issues you consider. Try to think outside the box and consider as many potential problems as possible. The scenario is: Planning a birthday party for a 5-year-old.

In [2]:
import pandas as pd
import json

# Load your CSV files into DataFrames
df_common_problem = pd.read_csv('raw/common_problem_task_prompt.csv')
df_consequences = pd.read_csv('raw/consequences_task_prompt.csv')
df_im = pd.read_csv('raw/im_task_prompt.csv')
df_is = pd.read_csv('raw/is_task_prompt.csv')
df_js = pd.read_csv('raw/js_task_prompt.csv')
df_situation = pd.read_csv('raw/situation_task_prompt.csv')
df_unusual = pd.read_csv('raw/unusual_task_prompt.csv')

# List of DataFrames and their names
dfs = [
    ('common_problem', df_common_problem),
    ('consequences', df_consequences),
    ('im', df_im),
    ('is', df_is),
    ('js', df_js),
    ('situation', df_situation),
    ('unusual', df_unusual)
]

# Initialize the list for JSON data
json_data = []

# Iterate over each DataFrame and create the JSON structure
for df_name, df in dfs:
    for idx, row in df.iterrows():

        entry = {
            "meta_data": {
                "dataset": "ttct",
                "id": f'{idx+1}_{df_name}',
                "eval_func": None
            },
            "input": {
                "text_basic": row['basic_prompt'],
                "text_instructive": row['instructive_prompt'],
                "text_cot": row['CoT_prompt'],
                "file": None,
                "others": {
                    "constraints": None,
                    "instructions": {
                        "system": "Do the following task or answer the following question.",
                        "user": "Instruction: {question}"
                    },
                    "references": {
                        "solutions": None
                    }
                }
            },
            "output": {
                "text_basic": None,
                "text_instructive": None,
                "text_cot": None
            }
        }
        
        # Append the entry to json_data
        json_data.append(entry)

# Save JSON file
with open('processed/ttct.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=4, ensure_ascii=False)

print("JSON file successfully created.")


JSON file successfully created.
